In [8]:
import numpy as np
import matplotlib.pyplot as plt
import random

def rand_posit(dim):
    return(random.randint(0,dim-1),random.randint(0,dim-1))

def in_posit(dim):
    return(random.randint(1,dim-2),random.randint(0,dim-2))

def diff_reg(dim, codes, nums):
    """
    anyone can make their own rules, but I hardcoded these in for a class:
        1=consumes the inhibitor 10% less
        2=consumes the activator 10% less
        3=overactive inhibitor 10% more
        4=overactive activator 10% more
    """
    if sum(nums)>0.5*dim**2:
        print("there are more anomalous cells than 50% of the cells available: try again")
        return
    Aout=np.ones((dim,dim))
    Iout=np.ones((dim,dim))
    A_outliers=[] #gather all the values to put into the matrix
    I_outliers=[]
    for i in range(0,len(codes)): #...there has to be a better way. 
        if codes[i]==1:
            I_outliers+=[0.9]*nums[i]
        if codes[i]==2:
            A_outliers+=[0.9]*nums[i]
        if codes[i]==3:
            I_outliers+=[1.1]*nums[i]
        if codes[i]==4:
            A_outliers+=[1.1]*nums[i]  
    for i in A_outliers: #place all randomly
        placed=False
        while placed==False:
            r,c=rand_posit(dim)
            if Aout[r,c]==1:
                Aout[r,c]=i
                placed=True
    for i in I_outliers: #place all randomly
        placed=False
        while placed==False:
            r,c=rand_posit(dim)
            if Iout[r,c]==1:
                Iout[r,c]=i
                placed=True
    return [Aout,Iout]

# ##### testing the old fashioned way: uncomment to see if function works. #####
# Aout,Iout=diff_reg(10,[1,2],[3,4])
# print(Aout,Iout)

def drop_cells(dim,num):
    """
    creates an array of num-number of randomly placed cells that do not have whatever Turing-patterning-program
    you convinced their kin to take up. Live free or die by the Puro! (although clearly they did not die by the puro:
    chances are this is one of those times when you transfect, select, then go into the experiment, YOLO--single cell
    cloning isn't real anyway, is it––Except oh. no. the landing pad is puro.  Okay. so use Blasticidin.  But also, you're going to 
    Spain in two weeks and Blast is SO slow.  So is this experiment worth doing, or can the doing or should you kill 
    everything (cells, hopes, dreams) before leaving.  Will 1% WT cells ruin everything? Find out tonight at 11.)
    """
    dropped=np.ones((dim,dim))
    for i in range(num):
        placed=False
        while placed==False:
            r,c=in_posit(dim)
            if dropped[r,c]==1:
                dropped[r,c]=0
                placed=True
    return dropped[1:-1,1:-1] #cut off borders

# ##### testing the old fashioned way: uncomment to see if function works. #####
# dropped=drop_cells(10,3)
# print(dropped)

In [17]:
##### bulk of the code #####

def setup(a_concn,i_concn,dim=100,uniform=True): #I have generally not used this, but in theory it'd be nice. 
    """
    makes the starting board, default starting size of 100
    if uniform=True, it takes a_concns and i_concns as a single number and applies it to all "cells", with a tiny bit of added noise.
    if uniform=False, a_concns and i_concns can be square numpy arrays with different values, but they must be the same size.  
    """
    if uniform: #needs noise!
        A=np.ones((dim,dim))*a_concn + np.random.rand(dim,dim)*0.01 
        I=np.ones((dim,dim))*i_concn + np.random.rand(dim,dim)*0.01
    else:
        if type(a_concn)!=np.array:
            print("concns not in correct format: needs to be numpy array")
        if size(a_concn)!=size(i_concn):
            print("check those sizes again.")
        A=a_concn
        I=i_concn
    return (A,I)

def laplacian(M,dx): 
    """
    takes square numpy array, M and space-step (dx). 
    calculates the laplacian and returns it 
    Note: the returned mtrix will have dimensions (N-2)x(N-2), as it lacks a border
    """
    return (M[0:-2, 1:-1]+M[1:-1, 0:-2]+M[2:, 1:-1]+M[1:-1, 2:]-4*(M[1:-1, 1:-1]))/(dx**2)

def fix_boundaries(M):
    """
    Takes a numpy array, M and matches the boundaries to their closest inner neighbor
    Returns the updated matrix
    """
    M[0,:]=M[1,:]
    M[-1,:]=M[-2,:]
    M[:,0]=M[:,1]
    M[:,-1]=M[:,-2]
    return(M)

def brusselator (U,Lu,V,Lv,Du,Dv,a,b,dt,drop=None):
    """
    U is a square numpy array representing concentrations of activator
    V is a square numpy array representing concentrations of inhibitor
    Lu and Lv are the laplacian of U and V, respectively
    Du and Dv are the diffusion constants for U and V, respectively
    a is the rate constant for the formation of the complex that "deactivates" the activator
    b is the rate constant for the creation of the inhibitor.
    dt is the timestep
    from "pattern formation in rxn diffusion" by physics programmers' youtube channel
    https://www.youtube.com/watch?v=khUMlMl8Vns
    """
    #ignore boundary
    u=U[1:-1,1:-1]
    v=V[1:-1,1:-1]
    #reaction term
    f=a-(1+b)*u+u**2*v
    g=b*u-u**2*v
    if drop:
        f=np.multiply(f,drop)
        g=np.multiply(g,drop)
    #update center to be: itself + reaction_term + diffusion_term
    U[1:-1, 1:-1]=u + dt*f + dt*Du*Lu
    V[1:-1, 1:-1]=v + dt*g + dt*Dv*Lv
    return(U,V)

    
def fitzhugh_nagumo (U,Lu,V,Lv,a,b,k,tau,dt,drop=None):
    """
    U is a square numpy array representing concentrations of activator
    V is a square numpy array representing concentrations of inhibitor
    Lu and Lv are the laplacian of U and V, respectively
    dt is the timestep size
    adapted from 
    Rossant, C. (2018). "12.4 Simulating a partial differential equation — reaction-diffusion systems and Turing patterns."" 
        In IPython interactive computing and visualization cookbook - second edition. Packt Publishing. 
    Originally used for membrane potential calculations; can be used to model RD rxns
    via Zheng and Shwen 2015 (https://www.sciencedirect.com/science/article/pii/S089812211500317X#br000185):
        "a is excitory threshold, e (or 1/tau) is excitability, b and k are "another parameter"
    """
    #ignore boundary
    u=U[1:-1,1:-1]
    v=V[1:-1,1:-1]
    #reaction term
    f=(u-u**3-v+k)
    g=(u-v)/tau
    if drop:
        f=np.multiply(f,drop)
        g=np.multiply(g,drop)
    #update inner
    U[1:-1,1:-1]=u + dt*f + dt*a*Lu
    V[1:-1,1:-1]=v + dt*g + dt*(b*Lv)/tau
    return(U,V)

def gray_scott(U,Lu,V,Lv,Du,Dv,a,b,dt,drop=None):
    """
    U is a square numpy array representing concentrations of activator
    V is a square numpy array representing concentrations of inhibitor
    Lu and Lv are the laplacian of U and V, respectively
    Du and Dv are the diffusion constants for U and V, respectively
    a is the rate at which the "activator" enters the system
    b is how quickly the "inhibitor" is depleted
    dt is the timestep. 
    Doelman, Kaper, Zegeling, 1997 Pattern formation in the one-dimensional Gray–Scott Model
        Nonlinearity 10 523, DOI 10.1088/0951-7715/10/2/013
    Garrity, M. (2015, March 16). How the tiger got its stripes. MathWorks Blogs. 
        Retrieved December 21, 2022, 
        from https://blogs.mathworks.com/graphics/2015/03/16/how-the-tiger-got-its-stripes/ 
    """
    #Ignore boundary
    u=U[1:-1,1:-1]
    v=V[1:-1,1:-1]
    #reaction term
    f=-u*v*v+a*(1-u)
    g=u*v*v-(a+b)*v
    if drop:
        f=np.multiply(f,drop)
        g=np.multiply(g,drop)
    #update
    U[1:-1,1:-1]=u + dt*f + dt*Du*Lu
    V[1:-1,1:-1]=v + dt*g + dt*Dv*Lv 
    return(U,V)

def gierer_meinhardt(U,Lu,V,Lv,Du,Dv,a,b,gamma,dt,drop=None):
    """
    from Nurahmi et al. 2018. 
    U is a square numpy array representing concentrations of activator
    V is a square numpy array representing concentrations of inhibitor
    Lu and Lv are the laplacian of U and V, respectively
    Du and Dv are the diffusion constants for U and V, respectively
    dt is the timestep size  
    a: production rate of activator
    b: production rate of inhibitor
    gamma scales the reaction
    
    Frankly, I see better versions where they split up Du and Dv, but following this for now 
    (parameters change when running with the last terms as Du*dt*Lu and Dv*dt*Lv)
    """
    #ignore boundary
    u=U[1:-1,1:-1]
    v=V[1:-1,1:-1]
    #reaction term
    f=a-b*u+u**2/v
    g=u**2-v
    if drop:
        f=np.multiply(f,drop)
        g=np.multiply(g,drop)
    #update
    d=Dv/Du #I like the versions where they keep these separate better...
    U[1:-1,1:-1]=u + dt*gamma*f + dt*Lu
    V[1:-1,1:-1]=v + dt*gamma*g + d*dt*Lv
    return(U,V)

def schnakenberg(U,Lu,V,Lv,Du,Dv,a,b,gamma,dt,drop=None): 
    """ 
    from Nurahmi et al. 2018. 
    based on mass action kinetics; assumes that the activator is silenced by binding with inhibitor
    accordingly, model is "trimolecular" if we take that complex to be a third species. 
    U is a square numpy array representing concentrations of activator
    V is a square numpy array representing concentrations of inhibitor
    Lu and Lv are the laplacian of U and V, respectively
    Du and Dv are the diffusion constants for U and V, respectively
    dt is the timestep size  
    a: production rate of activator
    b: production rate of inhibitor
    gamma scales the reaction
    """
    #ignore boundary
    u=U[1:-1,1:-1]
    v=V[1:-1,1:-1]
    #reaction term
    f=a-u+u*u*v
    g=b-u*u*v
    if drop:
        f=np.multiply(f,drop)
        g=np.multiply(g,drop)
    #update
    d=Dv/Du
    U[1:-1,1:-1]=u + dt*gamma*f + Du*dt*Lu
    V[1:-1,1:-1]=v + dt*gamma*g + Dv*dt*Lv
    return(U,V)


def run_and_plot(model,A,I,dx,dt,time,snapshot=10,const=[],off=None,drop=None):
    """
    model is a string, one of 'fn' for fitzhugh_nagumo, 's' for schnakenberg, or 'gm' for gierer_meinhardt,
        'b' for the brusselator, or 'gs' for gray-scott
    A and I boards with activator and inhibitor concentrations, respectively.  They can be made by hand, 
        or via the "setup" function
    dx is the spacestep size
    dt is the timestep size
    time is how long to run for
    snapshot is how many images you'd like to get out across the run
    const is the constants: models may differ in how many constants they require
        every model has a set of default constants. 
        Fitz-Nagumo requires
            a (position of fixed point)
            b (stability constant)
            tau: the ratio between the activator and inhibitor reactions
            k: external stimulus magnitude (the amoutn of activator added. )
        Gierer-Meinhardt requires
            diffusion activator
            diffusion inhibitor
            production rate of activator
            production rate of inhibitor
            scaling factor
        Schnkenberg requires
            diffusion activator
            diffusion inhibitor
            production rate of activator
            production rate of inhibitor
            scaling factor
        Gray-Scott requires
            diffusion activator
            diffusion inhibitor
            production rate of activator
            production rate of inhibitor
        Brusselator requires
            diffusion activator
            diffusion inhibitor
            formation rate for inhibition complex
            production rate of inhibitor
    """        
    numiter=int(time/dt)
    plotevery=numiter//snapshot
    
    for i in range(numiter+1):
        
        La=laplacian(A,dx)
        Li=laplacian(I,dx)
        
        if model=='fn':
            if const!=[]:
                a,b,tau,k=const[0],const[1],const[2],const[3]
            else:
                a,b,tau,k=2.8e-4,5e-3,0.1,-.005
            A,I=fitzhugh_nagumo(A,La,I,Li,a,b,k,tau,dt,drop)

        if model=='gm':
            if const!=[]:
                Da,Di,ka,ki,xn=const[0],const[1],const[2],const[3],const[4]
            else:
                Da,Di,ka,ki,xn=0.2,0.002,0.015,0.01,0.3
            A,I=gierer_meinhardt(A,La,I,Li,Da,Di,ka,ki,xn,dt,drop)
            
        if model=='s':
            if const!=[]:
                Da,Di,ka,ki,xn=const[0],const[1],const[2],const[3],const[4]
            else:
                Da,Di,ka,ki,xn=1,40,0.1,0.9,1
            A,I=schnakenberg(A,La,I,Li,Da,Di,ka,ki,xn,dt,drop)  
            
        if model=='gs':
            if const!=[]:
                Da,Di,a,b=const[0],const[1],const[2],const[3]
            else:
                Da,Di,a,b=1,0.5,0.055,0.062
            A,I=gray_scott(A,La,I,Li,Da,Di,a,b,dt,drop)
            
        if model=='b':
            if const!=[]:
                Du,Dv,a,b=const[0],const[1],const[2],const[3]
            else:
                Du, Dv, a, b=1,5,2,4
            A,I=brusselator(A,La,I,Li,Du,Dv,a,b,dt,drop)
    
        #deal with boundaries
        A=fix_boundaries(A)
        I=fix_boundaries(I)
        
        if off:
            A=np.multiply(A,off[0])
            I=np.multiply(I,off[1])
        
        #plot
        if i%plotevery==0 or i==numiter:
            plt.imshow(A,cmap=plt.cm.viridis, interpolation='bilinear') 
            plt.title('iter= '+str(i))
            if i==numiter:
                plt.title('final iter: '+str(i))
            plt.show() 
    return

In [19]:
##### Uncomment to run the models #####

# ##### S Model ##### 
# dim=100
# A=np.random.rand(dim,dim)
# I=np.random.rand(dim,dim)
# #constants from Maini PK, Woolley TE, Baker RE, Gaffney EA, Lee SS. Turing's model for biological pattern formation and the robustness problem. 
# #Interface Focus. 2012 Aug 6;2(4):487-96. doi: 10.1098/rsfs.2011.0113. Epub 2012 Feb 8. PMID: 23919129; PMCID: PMC3363041.
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,const=[1,40,0.1,0.9,1])#Da,Di,ka,ki,xn

# ##### GM Model: gradient version #####
# dim=10
# A=np.ones((dim,dim))*0.00001
# A[:,-1]=np.ones((1,dim))
# I=np.ones((dim,dim))*0.00001
# I[:,1]=np.ones((1,dim))
# #constants from Nurahmi A, Putra PS, Nuraini N, Investigation occurrences of turing pattern in Schnakenberg and Gierer-Meinhardt equation. 2018. 
# #AIP Conference Proceedings 1937, 020013 (2018); https://doi.org/10.1063/1.5026085
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,const=[0.025,0.8,0,1,0.01]) #Da,Di,ka,ki,xn

# ##### GM Model stripey #####
# dim=10
# A=np.ones((dim,dim))*0.00001
# A[:,5]=np.ones((1,dim))
# I=np.ones((dim,dim))*0.00001
# I[:,1]=np.ones((1,dim))
# #constants from Nurahmi A, Putra PS, Nuraini N, Investigation occurrences of turing pattern in Schnakenberg and Gierer-Meinhardt equation. 2018. 
# #AIP Conference Proceedings 1937, 020013 (2018); https://doi.org/10.1063/1.5026085
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,const=[0.025,0.8,0,1,1]) #Da,Di,ka,ki,xn

# ##### GM Model recreation spotty #####
# dim=25
# A=np.random.uniform(low=-0.2,high=0.2,size=(dim,dim))
# I=np.random.uniform(low=-0.2,high=0.2,size=(dim,dim))
# #constants from Nurahmi A, Putra PS, Nuraini N, Investigation occurrences of turing pattern in Schnakenberg and Gierer-Meinhardt equation. 2018. 
# #AIP Conference Proceedings 1937, 020013 (2018); https://doi.org/10.1063/1.5026085
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,const=[0.09,0.5,0,.1,0.5]) #Da,Di,ka,ki,xn

# ##### FN MODEL #####
# dim=100
# A = np.random.rand(dim, dim)
# I = np.random.rand(dim, dim)
# #constants from IPython interactive computing and visualization cookbook chapter 12.4
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10)

# ##### GS MODEL ##### takes forever. 
# dim=128
# A=np.ones((dim,dim))
# I=np.zeros((dim,dim))
# #constants from How the tiger got its stripes by M. Garrity on Mathworks blog. 
# I[51:65,51:70]=1
# run_and_plot('gs',A,I,dx=1,dt=0.025,time=5000,const=[1,0.5,0.055,0.062]) 

# ##### Brusselator #####
# dim=50
# dt=0.01
# time=150
# A=np.random.rand(dim,dim)
# I=np.random.rand(dim,dim)
# #from "pattern formation in rxn diffusion" video from physics programmers youtube: https://www.youtube.com/watch?v=khUMlMl8Vns
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4]) 


In [11]:
##### Trying with over and under-responses to activator and inhibitor #####

# ##### FN MODEL #####
# dim=100
# A = np.random.rand(dim, dim)
# I = np.random.rand(dim, dim)
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10,snapshot=1)
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10,snapshot=1,off=diff_reg(dim,[1],[100]))
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10,snapshot=1,off=diff_reg(dim,[2],[100]))
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10,snapshot=1,off=diff_reg(dim,[3],[100]))
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10,snapshot=1,off=diff_reg(dim,[4],[100]))

# ##### GM Model #####
# dim=10
# A=np.ones((dim,dim))*0.00001
# A[:,-1]=np.ones((1,dim))
# I=np.ones((dim,dim))*0.00001
# I[:,1]=np.ones((1,dim))
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.025,0.8,0,1,0.01]) #Da,Di,ka,ki,xn
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.025,0.8,0,1,0.01],off=diff_reg(dim,[1],[1]))
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.025,0.8,0,1,0.01],off=diff_reg(dim,[2],[1]))
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.025,0.8,0,1,0.01],off=diff_reg(dim,[3],[1]))
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.025,0.8,0,1,0.01],off=diff_reg(dim,[4],[1]))

# ##### GM Model #####
# dim=30
# A=np.random.uniform(low=-0.2,high=0.2,size=(dim,dim))
# I=np.random.uniform(low=-0.2,high=0.2,size=(dim,dim))
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.09,0.5,0,1,0.5])
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.09,0.5,0,1,0.5],off=diff_reg(dim,[1],[9]))
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.09,0.5,0,1,0.5],off=diff_reg(dim,[2],[9]))
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.09,0.5,0,1,0.5],off=diff_reg(dim,[3],[9]))
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.09,0.5,0,1,0.5],off=diff_reg(dim,[4],[9]))

# ##### S Model ##### 
# dim=100
# A=np.random.rand(dim,dim)
# I=np.random.rand(dim,dim)
# #constants from Maini PK, Woolley TE, Baker RE, Gaffney EA, Lee SS. Turing's model for biological pattern formation and the robustness problem. 
# #Interface Focus. 2012 Aug 6;2(4):487-96. doi: 10.1098/rsfs.2011.0113. Epub 2012 Feb 8. PMID: 23919129; PMCID: PMC3363041.
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,snapshot=1,const=[1,40,0.1,0.9,1])
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,snapshot=1,const=[1,40,0.1,0.9,1],off=diff_reg(dim,[2],[100]))
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,snapshot=1,const=[1,40,0.1,0.9,1],off=diff_reg(dim,[2],[100]))
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,snapshot=1,const=[1,40,0.1,0.9,1],off=diff_reg(dim,[2],[100]))
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,snapshot=1,const=[1,40,0.1,0.9,1],off=diff_reg(dim,[2],[100]))

# ##### Brusselator #####
# dim=50
# dt=0.01
# time=150
# A=np.random.rand(dim,dim)
# I=np.random.rand(dim,dim)
# #from "pattern formation in rxn diffusion" video from physics programmers youtube: https://www.youtube.com/watch?v=khUMlMl8Vns
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4]) 
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4],snapshot=1,off=diff_reg(dim,[1],[25])) 
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4],snapshot=1,off=diff_reg(dim,[2],[25])) 
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4],snapshot=1,off=diff_reg(dim,[3],[25])) 
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4],snapshot=1,off=diff_reg(dim,[4],[25])) 

In [12]:
##### "Dropped" cells: cells that do not have reaction capabilities #####

# ##### FN MODEL #####
# dim=100
# A = np.random.rand(dim, dim)
# I = np.random.rand(dim, dim)
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10,snapshot=1)
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10,snapshot=1,drop=[drop_cells(dim,100)])
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10,snapshot=1,drop=[drop_cells(dim,dim**2//10)])
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10,snapshot=1,drop=[drop_cells(dim,dim**2//2)])
# run_and_plot('fn',A,I,dx=2./len(A[1]),dt=0.001,time=10,snapshot=1,drop=[drop_cells(dim,(dim-2)**2)])

# ##### GM Model #####
# dim=10
# A=np.ones((dim,dim))*0.00001
# A[:,-1]=np.ones((1,dim))
# I=np.ones((dim,dim))*0.00001
# I[:,1]=np.ones((1,dim))
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.025,0.8,0,1,0.01]) #Da,Di,ka,ki,xn
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.025,0.8,0,1,0.01],drop=[drop_cells(dim,1)])
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.025,0.8,0,1,0.01],drop=[drop_cells(dim,dim**2//10)])
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.025,0.8,0,1,0.01],drop=[drop_cells(dim,dim**2//2)])
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.025,0.8,0,1,0.01],drop=[drop_cells(dim,(dim-2)**2)])

# ##### GM Model #####
# dim=30
# A=np.random.uniform(low=-0.2,high=0.2,size=(dim,dim))
# I=np.random.uniform(low=-0.2,high=0.2,size=(dim,dim))
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.09,0.5,0,1,0.5])
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.09,0.5,0,1,0.5],drop=[drop_cells(dim,9)])
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.09,0.5,0,1,0.5],drop=[drop_cells(dim,dim**2//10)])
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.09,0.5,0,1,0.5],drop=[drop_cells(dim,dim**2//2)])
# run_and_plot('gm',A,I,dx=1,dt=0.01,time=3,snapshot=1,const=[0.09,0.5,0,1,0.5],drop=[drop_cells(dim,(dim-2)**2)])

# ##### S Model ##### 
# dim=100
# A=np.random.rand(dim,dim)
# I=np.random.rand(dim,dim)
# #constants from Maini PK, Woolley TE, Baker RE, Gaffney EA, Lee SS. Turing's model for biological pattern formation and the robustness problem. 
# #Interface Focus. 2012 Aug 6;2(4):487-96. doi: 10.1098/rsfs.2011.0113. Epub 2012 Feb 8. PMID: 23919129; PMCID: PMC3363041.
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,snapshot=1,const=[1,40,0.1,0.9,1])
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,snapshot=1,const=[1,40,0.1,0.9,1],drop=[drop_cells(dim,100)])
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,snapshot=1,const=[1,40,0.1,0.9,1],drop=[drop_cells(dim,dim**2//10)])
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,snapshot=1,const=[1,40,0.1,0.9,1],drop=[drop_cells(dim,dim**2//2)])
# run_and_plot('s',A,I,dx=1,dt=.001,time=20,snapshot=1,const=[1,40,0.1,0.9,1],drop=[drop_cells(dim,(dim-2)**2)])

# ##### Brusselator #####
# dim=50
# dt=0.01
# time=150
# A=np.random.rand(dim,dim)
# I=np.random.rand(dim,dim)
# #from "pattern formation in rxn diffusion" video from physics programmers youtube: https://www.youtube.com/watch?v=khUMlMl8Vns
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4]) 
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4],snapshot=1,drop=[drop_cells(dim,25)]) 
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4],snapshot=1,drop=[drop_cells(dim,dim**2//10)]) 
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4],snapshot=1,drop=[drop_cells(dim,dim**2//2)]) 
# run_and_plot('b',A,I,dx=1,dt=dt,time=time,const=[1,5,2,4],snapshot=1,drop=[drop_cells(dim,(dim-2)**2)])

